# Fluidize-Python Interactive Demo

This notebook demonstrates the fluidize-python library for managing scientific computing projects.

## Setup

First, let's import the client and see where our projects will be stored:

In [4]:
# Import the fluidize client - handlers auto-register!
from fluidize.client import FluidizeClient
from fluidize.config import FluidizeConfig

# Create client and config
client = FluidizeClient(mode="local")
config = FluidizeConfig(mode="local")

print(f"📁 Projects will be stored in: {config.local_projects_path}")
print(f"📁 Base directory: {config.local_base_path}")
print(f"🚀 Client ready in '{client.mode}' mode!")

📁 Projects will be stored in: /Users/henrybae/.fluidize/projects
📁 Base directory: /Users/henrybae/.fluidize
🚀 Client ready in 'local' mode!


## 1. Creating Projects

Let's create some projects with different configurations:

In [16]:
# Create a comprehensive project
project1 = client.projects.create(
    project_id="data-pipeline-2024",
    label="Data Processing Pipeline",
    description="A comprehensive data processing pipeline for customer analytics",
    location="/projects/data/customer-analytics",
    status="active",
)

print("✅ Created project 1:")
print(f"   ID: {project1.id}")
print(f"   Label: {project1.label}")
print(f"   Status: {project1.status}")

✅ Created project 1:
   ID: data-pipeline-2024
   Label: Data Processing Pipeline
   Status: active


In [17]:
# Create a minimal project
project2 = client.projects.create(project_id="quick-experiment", label="Quick Experiment")

print("✅ Created project 2:")
print(f"   ID: {project2.id}")
print(f"   Label: {project2.label}")
print(f"   Status: {project2.status}")
print(f"   Description: '{project2.description}'")

✅ Created project 2:
   ID: quick-experiment
   Label: Quick Experiment
   Status: 
   Description: ''


## 2. Listing Projects

Let's see all the projects we have:

In [6]:
# Get all projects
projects = client.projects.list()

print(f"📋 Found {len(projects)} projects:")
print()

for i, project in enumerate(projects, 1):
    print(f"{i:2}. {project.id}")
    print(f"     Label: {project.label}")
    print(f"     Status: {project.status}")
    if project.description:
        print(f"     Description: {project.description[:50]}{'...' if len(project.description) > 50 else ''}")
    print()

📋 Found 1 projects:

 1. project-1754038373536
     Label: SIMPLETEST
     Status: active



## 3. Retrieving Specific Projects

Get detailed information about a specific project:

In [7]:
# Get project details
project = client.projects.get("project-1754038373536")

print("📊 Project Details:")
print(f"   ID: {project.id}")
print(f"   Label: {project.label}")
print(f"   Description: {project.description}")
print(f"   Status: {project.status}")
print(f"   Location: {project.location}")
print(f"   Metadata Version: {project.metadata_version}")

📊 Project Details:
   ID: project-1754038373536
   Label: SIMPLETEST
   Description: 
   Status: active
   Location: 
   Metadata Version: 1.0


In [12]:
from fluidize.core.types.runs import RunFlowPayload

payload = RunFlowPayload(
    name="simulation-run-1", description="Running simulation flow", tags=["simulation", "analysis"]
)


project.runs.run_flow(payload)

No start node provided, using first node: node-1754038461760
BFS traversal starting from node 'node-1754038461760':
  - Adding node to traversal: node-1754038461760, previous node: None
    - Adding neighbor to queue: node-1754038465820, will follow node-1754038461760
  - Adding node to traversal: node-1754038465820, previous node: node-1754038461760
Nodes to run: ['node-1754038461760', 'node-1754038465820']
Created project run folder: /Users/henrybae/.fluidize/projects/project-1754038373536/runs/run_3
Created run environment with number: 3


{'flow_status': 'running', 'run_number': 3}

Failed to log node parameters: 


Executing node node-1754038461760 in run 3

=== Starting run for node: node-1754038461760 ===
1. Preparing environment...
🔧 [Environment] Processing 0 targeted files (vs exhaustive search)
2. Executing simulation...


Failed to log node parameters: 


3. Handling files...
=== Run completed for node: node-1754038461760 with result: True ===

Executing node node-1754038465820 in run 3

=== Starting run for node: node-1754038465820 ===
1. Preparing environment...
🔧 [Environment] Processing 0 targeted files (vs exhaustive search)
2. Executing simulation...


No active MLFlow run to log metrics to
No active MLFlow run to log tags to


3. Handling files...
=== Run completed for node: node-1754038465820 with result: True ===



## 4. Updating Projects

Modify existing projects:

In [20]:
# Update project status and description
updated_project = client.projects.update(
    project_id="quick-experiment",
    status="in-progress",
    description="Testing various ML algorithms for classification",
    location="/experiments/ml/classification",
)

print("✅ Updated project:")
print(f"   ID: {updated_project.id}")
print(f"   Label: {updated_project.label}")
print(f"   Status: {updated_project.status}")
print(f"   Description: {updated_project.description}")
print(f"   Location: {updated_project.location}")

✅ Updated project:
   ID: quick-experiment
   Label: Quick Experiment
   Status: in-progress
   Description: Testing various ML algorithms for classification
   Location: /experiments/ml/classification


## 5. Working with Project Files

Let's explore the actual files created on disk:

In [21]:
import json

import yaml

# Get project directory
project_dir = config.local_projects_path / "data-pipeline-2024"
print(f"📁 Project directory: {project_dir}")
print(f"📁 Directory exists: {project_dir.exists()}")
print()

# List files in project
if project_dir.exists():
    print("📄 Files in project:")
    for file in project_dir.iterdir():
        print(f"   - {file.name} ({file.stat().st_size} bytes)")
    print()

📁 Project directory: /Users/henrybae/.fluidize/projects/data-pipeline-2024
📁 Directory exists: True

📄 Files in project:
   - parameters.json (40 bytes)
   - metadata.yaml (236 bytes)
   - graph.json (32 bytes)



In [22]:
# Read and display metadata file
metadata_file = project_dir / "metadata.yaml"
if metadata_file.exists():
    with open(metadata_file) as f:
        metadata = yaml.safe_load(f)

    print("📋 Project Metadata:")
    print(json.dumps(metadata, indent=2))
    print()

📋 Project Metadata:
{
  "project": {
    "description": "A comprehensive data processing pipeline for customer analytics",
    "id": "data-pipeline-2024",
    "label": "Data Processing Pipeline",
    "location": "/projects/data/customer-analytics",
    "metadata_version": "1.0",
    "status": "active"
  }
}



In [23]:
# Read graph and parameters files
graph_file = project_dir / "graph.json"
params_file = project_dir / "parameters.json"

if graph_file.exists():
    with open(graph_file) as f:
        graph_data = json.load(f)
    print("🔗 Graph Structure:")
    print(json.dumps(graph_data, indent=2))
    print()

if params_file.exists():
    with open(params_file) as f:
        params_data = json.load(f)
    print("⚙️  Parameters:")
    print(json.dumps(params_data, indent=2))

🔗 Graph Structure:
{
  "nodes": [],
  "edges": []
}

⚙️  Parameters:
{
  "metadata": {},
  "parameters": {}
}


## 6. Error Handling

Let's test error handling for non-existent projects:

In [24]:
# Try to get a non-existent project
try:
    missing_project = client.projects.get("non-existent-project")
    print(f"Found project: {missing_project.id}")
except FileNotFoundError:
    print("❌ Project 'non-existent-project' not found (expected behavior)")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

❌ Project 'non-existent-project' not found (expected behavior)


## 7. Batch Operations

Create multiple projects and work with them:

In [25]:
# Create multiple test projects
test_projects = [
    {
        "project_id": "ml-model-training",
        "label": "ML Model Training",
        "description": "Training deep learning models for image classification",
        "status": "training",
    },
    {
        "project_id": "data-preprocessing",
        "label": "Data Preprocessing",
        "description": "Cleaning and preparing raw data for analysis",
        "status": "active",
    },
    {
        "project_id": "model-evaluation",
        "label": "Model Evaluation",
        "description": "Evaluating and comparing different model performances",
        "status": "pending",
    },
]

created_projects = []
for project_data in test_projects:
    project = client.projects.create(**project_data)
    created_projects.append(project)
    print(f"✅ Created: {project.id} - {project.label}")

print(f"\n📊 Created {len(created_projects)} test projects")

✅ Created: ml-model-training - ML Model Training
✅ Created: data-preprocessing - Data Preprocessing
✅ Created: model-evaluation - Model Evaluation

📊 Created 3 test projects


In [26]:
# Filter projects by status
all_projects = client.projects.list()

# Group by status
by_status = {}
for project in all_projects:
    status = project.status or "unknown"
    if status not in by_status:
        by_status[status] = []
    by_status[status].append(project)

print("📊 Projects by Status:")
for status, projects in by_status.items():
    print(f"\n🏷️  {status.upper()}: {len(projects)} projects")
    for project in projects:
        print(f"   - {project.id}: {project.label}")

📊 Projects by Status:

🏷️  IN-PROGRESS: 1 projects
   - quick-experiment: Quick Experiment

🏷️  PENDING: 1 projects
   - model-evaluation: Model Evaluation

🏷️  ACTIVE: 2 projects
   - data-pipeline-2024: Data Processing Pipeline
   - data-preprocessing: Data Preprocessing

🏷️  TRAINING: 1 projects
   - ml-model-training: ML Model Training


## 8. Cleanup (Optional)

Delete test projects if needed:

In [27]:
# Uncomment and run this cell if you want to clean up test projects

# test_project_ids = ["ml-model-training", "data-preprocessing", "model-evaluation"]

# for project_id in test_project_ids:
#     try:
#         client.projects.delete(project_id)
#         print(f"🗑️ Deleted: {project_id}")
#     except FileNotFoundError:
#         print(f"❌ Project {project_id} not found")

print("💡 Uncomment the code above to delete test projects")

💡 Uncomment the code above to delete test projects


## 9. Working with Project Graphs

Now let's explore the new graph functionality integrated with projects:

In [28]:
# Import graph types
from fluidize.core.types.graph import GraphEdge, GraphNode, Position, graphNodeData

# Get our test project
project = client.projects.get("data-pipeline-2024")

print(f"📊 Working with project: {project.label}")
print(f"🔗 Graph access: {project.graph}")

# Get the current graph (should be empty initially)
graph_data = project.graph.get()
print("\n📈 Initial graph state:")
print(f"   Nodes: {len(graph_data.nodes)}")
print(f"   Edges: {len(graph_data.edges)}")

📊 Working with project: Data Processing Pipeline
🔗 Graph access: <fluidize.managers.project_graph.ProjectGraph object at 0x10a2d0990>

📈 Initial graph state:
   Nodes: 0
   Edges: 0


In [29]:
# Create some nodes for our data pipeline
node1_data = graphNodeData(
    label="Data Ingestion", simulation_id="data-ingest-v1", description="Load raw data from various sources"
)
node1 = GraphNode(id="data-ingestion-001", position=Position(x=100.0, y=100.0), data=node1_data, type="data-source")

node2_data = graphNodeData(
    label="Data Cleaning", simulation_id="data-clean-v1", description="Clean and validate data quality"
)
node2 = GraphNode(id="data-cleaning-001", position=Position(x=300.0, y=100.0), data=node2_data, type="processor")

node3_data = graphNodeData(
    label="Feature Engineering", simulation_id="feature-eng-v1", description="Extract and transform features"
)
node3 = GraphNode(id="feature-engineering-001", position=Position(x=500.0, y=100.0), data=node3_data, type="processor")

# Add nodes to the project graph
print("🔹 Adding nodes to project graph:")
project.graph.add_node(node1)
print(f"   ✅ Added: {node1.data.label}")

project.graph.add_node(node2)
print(f"   ✅ Added: {node2.data.label}")

project.graph.add_node(node3)
print(f"   ✅ Added: {node3.data.label}")

# Check updated graph
graph_data = project.graph.get()
print("\n📈 Updated graph state:")
print(f"   Nodes: {len(graph_data.nodes)}")
for node in graph_data.nodes:
    print(f"   - {node.id}: {node.data.label} ({node.type})")

🔹 Adding nodes to project graph:
   ✅ Added: Data Ingestion
   ✅ Added: Data Cleaning
   ✅ Added: Feature Engineering

📈 Updated graph state:
   Nodes: 3
   - data-ingestion-001: Data Ingestion (data-source)
   - data-cleaning-001: Data Cleaning (processor)
   - feature-engineering-001: Feature Engineering (processor)


In [ ]:
# Create edges to connect the pipeline nodes
edge1 = GraphEdge(id="ingest-to-clean", source="data-ingestion-001", target="data-cleaning-001", type="data-flow")

edge2 = GraphEdge(id="clean-to-feature", source="data-cleaning-001", target="feature-engineering-001", type="data-flow")

# Add edges to connect the workflow
print("🔗 Adding edges to create workflow:")
project.graph.add_edge(edge1)
print("   ✅ Connected: Data Ingestion → Data Cleaning")

project.graph.add_edge(edge2)
print("   ✅ Connected: Data Cleaning → Feature Engineering")

# Check final graph state
graph_data = project.graph.get()
print("\n📊 Final pipeline graph:")
print(f"   Nodes: {len(graph_data.nodes)}")
print(f"   Edges: {len(graph_data.edges)}")

print("\n🔗 Workflow connections:")
for edge in graph_data.edges:
    source_node = next(n for n in graph_data.nodes if n.id == edge.source)
    target_node = next(n for n in graph_data.nodes if n.id == edge.target)
    print(f"   {source_node.data.label} → {target_node.data.label}")

## 10. Graph Operations

Let's demonstrate updating and managing the graph:

In [ ]:
# Update node position
updated_node = GraphNode(
    id="feature-engineering-001",
    position=Position(x=500.0, y=200.0),  # Move it down
    data=node3_data,
    type="processor",
)

print("📍 Updating node position:")
result = project.graph.update_node_position(updated_node)
print(f"   ✅ Moved {result.data.label} to ({result.position.x}, {result.position.y})")

# Demonstrate deletion (we'll add the node back)
print("\n🗑️ Demonstrating node deletion:")
original_graph = project.graph.get()
print(f"   Before deletion: {len(original_graph.nodes)} nodes, {len(original_graph.edges)} edges")

project.graph.delete_node("feature-engineering-001")
after_deletion = project.graph.get()
print(f"   After deletion: {len(after_deletion.nodes)} nodes, {len(after_deletion.edges)} edges")
print("   ⚠️ Note: Connected edges are automatically removed!")

# Add the node back
print("\n🔄 Adding node back:")
project.graph.add_node(node3)
restored_graph = project.graph.get()
print(f"   After restore: {len(restored_graph.nodes)} nodes, {len(restored_graph.edges)} edges")
print("   ⚠️ Note: Need to manually reconnect edges!")

## 11. Graph Isolation & Persistence

Demonstrate that graphs are isolated per project and persist to disk:

In [ ]:
# Get another project to show isolation
experiment_project = client.projects.get("quick-experiment")

print("🔬 Working with different project:")
print(f"   Project 1: {project.label} - {len(project.graph.get().nodes)} nodes")
print(f"   Project 2: {experiment_project.label} - {len(experiment_project.graph.get().nodes)} nodes")

# Add a node to the experiment project
experiment_node = GraphNode(
    id="experiment-node-001",
    position=Position(x=0.0, y=0.0),
    data=graphNodeData(
        label="ML Model Training", simulation_id="ml-train-v1", description="Training a neural network model"
    ),
    type="ml-model",
)

experiment_project.graph.add_node(experiment_node)

print("\n📊 After adding node to experiment project:")
print(f"   Project 1: {project.label} - {len(project.graph.get().nodes)} nodes")
print(f"   Project 2: {experiment_project.label} - {len(experiment_project.graph.get().nodes)} nodes")
print("   ✅ Graphs are completely isolated!")

In [ ]:
# Check graph persistence on disk
import json

project_dir = config.local_projects_path / "data-pipeline-2024"
graph_file = project_dir / "graph.json"

print("💾 Graph persistence on disk:")
print(f"   Graph file: {graph_file}")
print(f"   File exists: {graph_file.exists()}")

if graph_file.exists():
    with open(graph_file) as f:
        graph_json = json.load(f)

    print("   📄 Graph file contents:")
    print(f"      Nodes in file: {len(graph_json.get('nodes', []))}")
    print(f"      Edges in file: {len(graph_json.get('edges', []))}")

    if graph_json.get("nodes"):
        print("      Node examples:")
        for node in graph_json["nodes"][:2]:  # Show first 2 nodes
            print(f"        - {node.get('id', 'unknown')}: {node.get('data', {}).get('label', 'no label')}")

print("\n🔄 Graph data matches in-memory state:")
current_graph = project.graph.get()
print(f"   In-memory nodes: {len(current_graph.nodes)}")
print(f"   In-memory edges: {len(current_graph.edges)}")
print("   ✅ Data is automatically persisted to disk!")

## 12. Your Experiments

Use the cells below for your own experiments with projects and graphs:

In [ ]:
# Your code here - experiment with the fluidize client!
# Try creating your own projects and building graphs:
#
# project = client.projects.create(project_id="my-experiment", label="My Experiment")
# node = GraphNode(
#     id="my-node",
#     position=Position(x=100.0, y=100.0),
#     data=graphNodeData(label="My Node", simulation_id="my-sim"),
#     type="custom"
# )
# project.graph.add_node(node)
#
# Explore the API!

In [ ]:
# More experimentation space - try advanced graph operations:
#
# # Build complex workflows
# # Update node positions
# # Connect multiple projects
# # Test error handling
#
# print("Ready for your experiments!")

## Summary

This notebook demonstrated the complete fluidize-python API:

### ✅ **Project Management**
- **Simple Import** - Just `from fluidize.client import FluidizeClient`  
- **Project CRUD Operations** - Create, Read, Update, Delete  
- **File System Integration** - Projects stored in `~/.fluidize/projects/`  
- **Error Handling** - Proper exceptions for missing projects  
- **Batch Operations** - Working with multiple projects  

### ✅ **New Graph Integration** 
- **Intuitive API** - `project.graph.add_node(node)` instead of complex backend calls
- **Project Scoping** - All graph operations automatically scoped to project
- **Node & Edge Operations** - Add, update, delete nodes and edges
- **Graph Persistence** - Automatic saving to `graph.json` 
- **Cross-Project Isolation** - Each project has its own independent graph
- **Position Management** - Update node positions with `update_node_position()`
- **Smart Edge Cleanup** - Deleting nodes automatically removes connected edges

### ✅ **User-Friendly Design**
- **No Context Passing** - `project.graph.operation()` instead of `backend.graph.operation(project, ...)`
- **Automatic Persistence** - Changes immediately saved to disk
- **Type Safety** - Full Pydantic model validation
- **Clean Architecture** - Project wrapper with lazy-loaded graph property

**Next Steps:**
- Build complex computational workflows with the graph API
- Integrate with simulation runners (coming soon)  
- Connect to cloud APIs (API mode)
- Scale to distributed computing environments